In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies=pd.read_csv('../../data/data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(3)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [4]:
movies_df=movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [5]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [6]:
from ast import literal_eval # String --> Real Data Type (dict)
movies_df['genres']=movies_df['genres'].apply(literal_eval)
movies_df['keywords']=movies_df['keywords'].apply(literal_eval)

In [16]:
movies_df['genres']=movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [18]:
movies_df['genres'][0][0]

'Action'

In [27]:
('/').join(['TMDB','MOVIE','DATASET'])

'TMDB/MOVIE/DATASET'

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

movies_df['genres_literal']=movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0,ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [33]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim=cosine_similarity(genre_mat,genre_mat)
print(genre_sim.shape)
print(genre_sim)

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [38]:
a=np.array([[1,2,3],
            [2,1,7],
            [91,0,9]])
np.argsort(a)  ######## Return Index #######

array([[0, 1, 2],
       [1, 0, 2],
       [1, 2, 0]], dtype=int64)

In [42]:
genre_sim_sorted_ind=genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])  # 3494 row 와  코사인 유사도가 가장 높다. 

[[   0 3494  813 ... 3038 3037 2401]]


In [43]:
def find_sim_movie(df,sorted_ind,title_name,top_n=10):
    title_movie= df[df['title']==title_name] 
    
    title_index= title_movie.index.values # ndarray
    similar_indexes = sorted_ind[title_index,:(top_n)]
    
    print(similar_indexes)
    similar_indexes= similar_indexes.reshape(-1) # 1D ndarray
    return df.iloc[similar_indexes]
    
    
    

In [45]:
similar_movies=find_sim_movie(movies_df,genre_sim_sorted_ind,'The Godfather',10)
similar_movies[['title','vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [60]:
find_sim_movie(movies_df,genre_sim_sorted_ind,'Edge of Tomorrow',10)[['title','vote_average']]

[[ 854 3089   74 1650 1270 4012  634  582 4176 4187]]


,title,vote_average
854,Æon Flux,5.4
3089,Krrish,5.5
74,Edge of Tomorrow,7.6
1650,Wing Commander,4.0
1270,Universal Soldier: The Return,4.2
4012,Escape from the Planet of the Apes,6.3
634,The Matrix,7.9
582,Battle: Los Angeles,5.5
4176,Battle for the Planet of the Apes,5.5
4187,Independence Daysaster,4.2


In [61]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [62]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3),'m:',round(m,3))

C: 6.092 m: 370.2


In [64]:
percentile= 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m))*R) + ((m/(v+m))*C)

movies_df['weighted_vote']= movies_df.apply(weighted_vote_average,axis=1)

In [66]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [67]:
def find_sim_movie(df,sorted_ind,title_name,top_n=10):
    title_movie= df[df['title']==title_name] 
    
    title_index= title_movie.index.values # ndarray
    similar_indexes = sorted_ind[title_index,:(top_n*2)]
    similar_indexes= similar_indexes.reshape(-1) # 1D ndarray
    
    similar_indexes=similar_indexes[similar_indexes != title_index]
    
    
    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]

In [68]:
find_sim_movie(movies_df,genre_sim_sorted_ind,'Edge of Tomorrow',10)[['title','vote_average']]

,title,vote_average
2912,Star Wars,8.1
634,The Matrix,7.9
2253,Equilibrium,6.9
266,"I, Robot",6.7
257,Real Steel,6.6
415,Dredd,6.6
1608,The Running Man,6.4
1153,Lucy,6.3
223,The Chronicles of Riddick,6.3
4012,Escape from the Planet of the Apes,6.3
